# Synthetic Evaluation Data Generation


## Quickstart

### Install required libraries

```
$ pip install -r requirements.txt
```

Please also see [README.md](../README.md) for environment setup including necessary library installation.


### Prepare input data

The synthetic data generation framework supports two input formats `rawdoc` or `squad`. 

- `input_format=rawdoc`

The file should be stored in a JSONL format. Each line contains a document in the format of `{"text": <document>, "title": <title>}`.

```
{"text": "The quick brown fox jumps over the lazy dog.", "title": "Classic Pangram" }
{"text": "The Eiffel Tower is an iron lattice tower on the Champ de Mars in Paris.", "title": "Iconic Landmark" }
...
```
Additionally, if the documents already have a document id, the input file can also contain document ids. The same ids will be persisted in the generated data as well. Another accepted format is `{"_id": <document_id>, "text": <document>, "title": <title>}`.
```
{"_id": "5", "text": "The quick brown fox jumps over the lazy dog.", "title": "Classic Pangram" }
{"_id": "doc3", "text": "The Eiffel Tower is an iron lattice tower on the Champ de Mars in Paris.", "title": "Iconic Landmark" }
...
```
This repository contains a sample JSONL file `data/sample_data.jsonl`.


- `input_format=squad`

If you have manually created questions and would like to conduct further analysis (correlation between synthetic questions and original questions), the input data should follow the SQuAD format.

```
       {
            "data": [
                {
                    "paragraphs": [
                        {
                            "context": "The quick brown fox jumps over the lazy dog.",
                            "qas": [
                                {
                                    "question": "What does the fox jump over?",
                                    "id": "q1",
                                    "synthetic": true,
                                    "answers": [
                                        {
                                            "text": "The fox jump over the lazy dog",
                                            "answer_start": -1,  # For generative answers
                                            "synthetic": true,
                                        }
                                    ]
                                }
                            ]
                        }
                    ],
                    "title": "Example"
                }
            ],
            "version": "2.0"
        }        
```

In [1]:
import os
from omegaconf import OmegaConf
import sys
sys.path.append("../")
sys.path.append("../../../")
from retriever_evalset_generator import RetrieverEvalSetGenerator
from filters import EasinessFilter, AnswerabilityFilter
from nemo_curator.modules.filter import ScoreFilter, Score
from nemo_curator.datasets import DocumentDataset
from nemo_curator.modules.config import RetrieverEvalSDGConfig
import warnings
warnings.filterwarnings('ignore')


/home/vinay/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Generating API key

- The SDG pipeline uses NIM models, in order to use them, you need to generate an API key.

- Visit [this page](https://build.nvidia.com/mistralai/mixtral-8x7b-instruct) and click "Get API Key" to generate an API key

![NVIDIA API Catalog](../figures/api_key.png) 

### Loading datasets
We now load a sample dataset from out data folder

In [6]:
import pandas as pd
df = pd.read_json("../data/sample_data_rawdoc.jsonl", lines=True)

In [7]:
df.head()

,text,title,_id
0,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,NaN
1,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,NaN
2,The Taj Mahal is an ivory-white marble mausole...,Taj Mahal - A Symbol of Love,NaN
3,Machu Picchu is a 15th-century Inca citadel si...,Machu Picchu - Lost City of the Incas,NaN
4,"The Colosseum, also known as the Flavian Amphi...",The Colosseum - Ancient Roman Architecture,NaN


### Read pipeline config file, instantiate Generator Object

In [9]:
cfg = RetrieverEvalSDGConfig.from_yaml("../config/config.yaml")
# cfg.api_key = "your api key here"
retrieval_evalset_generator = RetrieverEvalSetGenerator(cfg)

### Running the Synthetic Data Generator
We first create the dataset object from the pandas dataframe, and pass along the dataset object through the generator and the filters. The dataset object gets transformed along the different steps of the pipeline (i.e. generator, filters)

In [10]:
dataset = DocumentDataset.from_pandas(df)
generated_dataset = retrieval_evalset_generator(dataset)
generated_df = generated_dataset.df.compute()

### Probing the generated Data
For those documents that do not have a document id, the pipeline generates a random hash as document id. For those that have an existing document id, the pipeline persists the same ids in the generated data.

In [16]:
generated_df.head()

,_id,text,title,question-id,question,answer,score
0,0549b6a6e85511ca5b3386a00b5d654993ed031c47dd23...,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,838fdf6da376a28a0e833f6a26771939c66773256d958e...,What is the significance of the Eiffel Tower i...,The Eiffel Tower was built to celebrate the 10...,1
1,eac01874c8696111bb2162539417bd228ba73ba179806a...,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,5aa0b9c8a29ed5bf50bf3830ae4eb98359d3771251f9d5...,Who was responsible for designing the Eiffel T...,The Eiffel Tower was designed by the engineer ...,1
2,f818910504ea6dcd9fd5299d62a0c504c7d5746c0e4b9a...,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,086919c222e3aa4a80af9837bccb08c1dc19c42bfd65b8...,When was the Eiffel Tower constructed and for ...,The Eiffel Tower was built in 1889 for the Exp...,1
3,b6b70ead39a706ebfab3957b57bceec19f57b00bf54bbf...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,602ff5c02c9c43fb83fa79048bf9d3374ed161b2f4298a...,What is the purpose of the Great Wall of China?,The purpose of the Great Wall of China is to p...,1
4,d395de2f6db8a0bc61c502b03f66746a8c78d2ea1068d1...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,7e9e58ea0b7550e80ab357a206a8198c83575322604149...,What materials were used to build the Great Wa...,The Great Wall of China was built using materi...,1


### Data Quality Assessment
We apply two filters:

*Answerability filer* uses LLM-as-judge in order to determine quality of questions in terms of them being answerable from content in the passage. The filter weeds out questions that are invalid and not relevant to the document chunk that was used to generate them.

*Easiness filter* is used to filter out questions that are deemed easy for the retriever models to retrieve positive passages for the given generated question. It uses embedding model as judge. The user needs to provide threshold (number between 0 and 1) for this filter. Lower the value of the filter, harder the questions in the dataset. If the threshold value is higher, then we have many easy questions in the dataset. 

The filters can be applied in any order. 

In [17]:
ef = EasinessFilter(cfg)
easiness_filter = ScoreFilter(ef,
                              text_field = ["text", "question"],
                              score_field = "easiness_scores")
af = AnswerabilityFilter(cfg)
answerability_filter = ScoreFilter(af,
                              text_field = ["text", "question"],
                              score_field = "answerability_scores")

### Easiness filter
We see an additional column being generated "easiness_scores". This filter removes questions that are too easy to retrieve by retriever models.

In [18]:
filtered_dataset = easiness_filter(generated_dataset)
filtered_df_1 = filtered_dataset.df.compute()

In [19]:
filtered_df_1.head()

,_id,text,title,question-id,question,answer,score,easiness_scores
1,ed03487395aec925207a6fad785f91fcb9cc4cf0c650dc...,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,7aa5c14a65dc512ed6b5cb43bbb256e0325148e782cb7c...,Who was responsible for designing the Eiffel T...,The Eiffel Tower was designed by the engineer ...,1,0.570877
2,1cc910b8300aa7ad2715f1947f37f86610fdc741a1f22a...,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,9ff5907351a6ce15bb5178bfc14d9d64ad1313aed816fb...,What event prompted the construction of the Ei...,The Eiffel Tower was built for the 1889 Exposi...,1,0.564394
3,c5bb45480854730c328b8311a5adf719333feb8f9a1aa3...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,7fe7cce49e4a472345068967f12e4ad3376599c0ac6929...,What materials were used to construct the Grea...,The Great Wall of China was constructed using ...,1,0.553244
4,77837bd4ea33b9e221b05e8d03765495179c3bd2114b8a...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,2e01d5c37f3bf1284d6d6fb60b2fd2cbf174ac41aec6c1...,What was the primary purpose of building the G...,The primary purpose of building the Great Wall...,1,0.505572
5,3e864422003c1f770a0b26d89b3f4c85a8d69b22a27768...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,5d398dadcbbc57d33c5ed86b8869d23f5d3a006f349556...,Which direction was the Great Wall of China ge...,The Great Wall of China was generally built al...,1,0.546477


In [21]:
print (f"Total number of generated data points = {generated_df.shape[0]}") 
print (f"Total number of data points after application of easiness filter = {filtered_df_1.shape[0]}")

Total number of generated data points = 60
Total number of data points after application of easiness filter = 42


### Answerability filter
We see additional column "answerability scores", which shows the rating provided by the LLM-as-judge on criteria used to judge the questions. The criteria can be found in the config. 

In [22]:
filtered_dataset_2 = answerability_filter(filtered_dataset)
filtered_df_2 = filtered_dataset_2.df.compute()

In [23]:
filtered_df_2.head()

,_id,text,title,question-id,question,answer,score,easiness_scores,answerability_scores
2,1141c4bbc2ece8a7bac3aa581f7a8b006b860abc73b95c...,The Eiffel Tower is an iconic landmark of Pari...,Eiffel Tower - A French Icon,e05dcf001cf8a151df7cfbc85ed9276302e188c42186ca...,What event prompted the construction of the Ei...,The Eiffel Tower was built for the 1889 Exposi...,1,0.552801,ChatCompletion(id='cmpl-c6630b460d05454ab1867f...
3,6e807d0d8d6e95fa553106d1904d0792e7abb7fb8ab256...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,589a845cbf8e35430aab2e85c7a7571eefeb230247ab8f...,What is the purpose of the Great Wall of China?,The purpose of the Great Wall of China is to p...,1,0.528075,ChatCompletion(id='cmpl-edd7fa04caf34b46bfb38d...
4,6eed572a0452044e4ae7e81e19c110bd265e2e82172ae5...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,7bd1461271ce20c9693e6dee66ab17d08457f2c9fdbdcc...,What materials were used to build the Great Wa...,The Great Wall of China was built using materi...,1,0.550626,ChatCompletion(id='cmpl-8c840245755b4f33b556f9...
5,f66a68e05a0075e5a764eb28046d2d4a4dec6dba6a5d5f...,The Great Wall of China is a series of fortifi...,The Great Wall of China - Ancient Protection,a6e6be7260737442c6b725a98b32f601b02abac2015728...,Which direction does the Great Wall of China g...,The Great Wall of China generally follows an e...,1,0.502974,ChatCompletion(id='cmpl-4e613107ea964ccd92d002...
6,38139c0792abce3e683f2e23111933e3cab7e43c43ac84...,The Taj Mahal is an ivory-white marble mausole...,Taj Mahal - A Symbol of Love,b84758ac3e57d133574a2b7407e8053c6921c9ac5c2956...,What is the Taj Mahal and who built it?,The Taj Mahal is an ivory-white marble mausole...,1,0.521754,ChatCompletion(id='cmpl-ca1fee27a87c405d883c20...


In [24]:
print (f"Total number of data points after application of answerability filter = {filtered_df_2.shape[0]}")

Total number of data points after application of answerability filter = 42


We see that upon adding the answerability filter, the number of data points did not reduce further. For this dataset post easiness filter, we have all answerable questions.